In [ ]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adtk.detector import AutoregressionAD
from adtk.data import validate_series

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["dadosVivamente"]
collection = db["postsAgregadoComRespostas"]

In [ ]:
# Filtrando documentos com o atributo maior ou igual a 2
filtro = {"suicida": {"$eq": "3"}}
documentos = collection.find(filtro)

In [ ]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))

In [ ]:
# Visualizando o DataFrame
df.head()

In [ ]:
df['postCreated_time'] = pd.to_datetime(df['postCreated_time'])
df.head()

In [ ]:
df['data'] = df['postCreated_time'].dt.to_period('M').dt.to_timestamp()
df.head()

In [ ]:
df_grouped_data = df.groupby(['id_usuario', 'data']).size().reset_index(name='quantidade')
# Substituir o NaN por 0
df_grouped_data['quantidade'] = df_grouped_data['quantidade'].fillna(0)

In [ ]:
print(df.dtypes)

In [ ]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

In [ ]:
print(df.dtypes)

In [ ]:
# Gerar o gráfico de anomalias de cada usuário
for usuario_id in df_grouped_data['id_usuario'].unique():
    df_usuario = df_grouped_data[df_grouped_data['id_usuario'] == usuario_id]

    # Salvar em csv
    # df_usuario.to_csv(f'./dados/{usuario_id}.csv', index=False)

    # Gerar o gráfico
    df_usuario.set_index('data', inplace=True)

    # Extrair a série temporal 'quantidade'
    s = df_usuario['quantidade']

    # Validar a série temporal
    s = validate_series(s)

    # Usar AutoregressionAD para detecção de anomalias
    autoreg_ad = AutoregressionAD(n_steps=3, step_size=1, c=1.5)

    # Aqui passamos a série temporal 's' para o método fit_detect
    anomalies = autoreg_ad.fit_detect(s)
    
    # Passo 4: Plotar os resultados
    plt.figure(figsize=(14, 7))

    # Plotar a série temporal
    sns.lineplot(x=s.index.astype(str), y=s.values, label='Quantidade de Posts')

    # Plotar as anomalias
    anomaly_points = s[anomalies == True]
    sns.scatterplot(x=anomaly_points.index.astype(str), y=anomaly_points.values, color='red', s=100, label='Anomalias')

    # Configurar o gráfico
    plt.title(f'Usuário {usuario_id} - Quantidade de Posts por Mês/Ano')
    plt.xlabel('Mês/Ano')
    plt.ylabel('Quantidade de Posts')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()

    # Mostrar o gráfico
    plt.show()